In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

from sklearn.metrics import f1_score

# Configuration

In [ ]:
# Define some hyperparameters
base_model = 'jplu/tf-xlm-roberta-large'
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 10

# Labels in our dataset.
labels = ["different", "similar"]

# Data Preparation

In [ ]:
# Specify input file location
training_file = '../input/product-matching-id-ndsc-2020/new_training_set.csv'
test_file = '../input/product-matching-id-ndsc-2020/new_test_set.csv'

# Data Understanding

In [ ]:
# Load dataset
training_data = pd.read_csv(training_file)
test_data = pd.read_csv(test_file)

In [ ]:
# Show sample training data 
training_data.head()

In [ ]:
# Show number of samples in training data
training_data.shape[0]

In [ ]:
# Show distribution of data
training_data.Label.value_counts()

In [ ]:
# Check for any missing data
training_data.isnull().sum()

In [ ]:
# Show sample of test data
test_data.head()

In [ ]:
# Show number of samples in test data
test_data.shape[0]

In [ ]:
# Check for any missing data
test_data.isnull().sum()

# Data Preparation

In [ ]:
# Take only samples without missing data
training_data = training_data[training_data['title_2'].notna()]

In [ ]:
# Convert the label to one-hot encoding format
y_train = tf.keras.utils.to_categorical(training_data.Label[:8000], num_classes=2)
y_validation = tf.keras.utils.to_categorical(training_data.Label[8000:], num_classes=2)

In [ ]:
# Define data generator
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            base_model, do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

# Modeling

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained model.
    pretrained_model = transformers.TFAutoModel.from_pretrained(base_model)
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    pretrained_model.trainable = False

    sequence_output, pooled_output = pretrained_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

In [ ]:
# Generate training, validation, and test set
training_set = BertSemanticDataGenerator(
    training_data[["title_1", "title_2"]].values.astype("str")[:8000],
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
validation_set = BertSemanticDataGenerator(
    training_data[["title_1", "title_2"]].values.astype("str")[8000:],
    y_validation,
    batch_size=batch_size,
    shuffle=False,
)

# Model Training

In [ ]:
# Train the model
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

# Generate Prediction

In [ ]:
# Generate prediction based on validation data
p_validation = model.predict(validation_set)
p_validation = np.argmax(p_validation, axis=1)

# Evaluate the prediction using F1 score
f1_score(np.argmax(y_validation[:len(p_validation)], axis=1), p_validation)

In [ ]:
# Generate test set
test_set = BertSemanticDataGenerator(
    test_data[["title_1", "title_2"]].values.astype("str"), 
    labels=None, 
    batch_size=20, 
    shuffle=False, 
    include_targets=False
)

# Generate prediction based on test data
prediction = model.predict(test_set)
prediction = np.argmax(prediction, axis=1)

In [ ]:
# Append the prediction to test data
test_data['label'] = prediction

In [ ]:
# Generate submission data
test_data.loc[:, ['label']].to_csv(
    'submission.csv', index=True, header=True
)